Oceanography python bootcamp, Winter 2025
# Week 5 notebook

In [ ]:
import os
import pytz
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import cmocean.cm as cmo
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.gridliner as cmgrid

pd.set_option("mode.copy_on_write", True)

## Navigating filesystem

In [ ]:
# Get and print the current directory
cwd = os.getcwd()
print(cwd)

In [ ]:
# list the content under the working directory
os.listdir()

In [ ]:
# list the FILES under the working directory
files = []
for sub in os.listdir():
    if os.path.isfile(sub):
        files.append(sub)
files

In [ ]:
# check if a file exists (useful check before write!)
file = "crazy.txt"
os.path.exists(file)

## Loading and inspecting data in pandas

### Reading data

Data obtained from https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USW00024233/detail

In [ ]:
# reading offline data
# interpret the "DATE" column as date
df_seatac = pd.read_csv("data/NOAA_SeaTac_weather_2024.csv", parse_dates = ["DATE"]).convert_dtypes()

In [ ]:
df_seatac

In [ ]:
# making use of the usecols and index_col arguments

df_seatac2 = pd.read_csv(
    "data/NOAA_SeaTac_weather_2024.csv", 
    usecols = ["DATE", "TAVG", "TMIN", "TMAX"],
    index_col= "DATE",
    parse_dates = ["DATE"]
).convert_dtypes()

In [ ]:
df_seatac2

Data obtained from https://github.com/allisonhorst/palmerpenguins

In [ ]:
# reading online data
# set Year as an index, skip the top row
df_penguins = pd.read_csv(
    "https://github.com/allisonhorst/palmerpenguins/blob/main/inst/extdata/penguins.csv?raw=true"
).convert_dtypes()

In [ ]:
df_penguins

### Inspecting data

In [ ]:
df_seatac.info()

In [ ]:
df_penguins.info()

In [ ]:
# first few rows of the dataframe
# also consider using df.tail()
df_seatac.head()

In [ ]:
# random sample from the dataframe
df_seatac.sample(5)

### Extracting parts of the DataFrame

In [ ]:
# get the Series of a single column of data
df_seatac["DATE"]

In [ ]:
# Get the collection of columns
df_seatac.columns

In [ ]:
df_seatac.index

In [ ]:
df_seatac.index.values

----

_**Exercise #2**_: Loading and inspecting data in pandas

+ Download the Seattle water levels data for yesterday from [https://tidesandcurrents.noaa.gov/waterlevels.html?id=9447130](https://tidesandcurrents.noaa.gov/waterlevels.html?id=9447130) as a csv file
+ Place the csv file in a convenient place and load it using pandas, include all but the verified column, and assign the resulting DataFrame to the variable `df_tides`
+ Display 5 random rows from `df_tides`
+ Find the number of rows in `df_tides`

In [ ]:
# part 2: load the csv file into a pandas DataFrame

In [ ]:
# part 3: display 5 random rows from df_tides

In [ ]:
# part 4: find the number of rows in df_tides

## Data wrangling in pandas

### .loc[] and .iloc[]

In [ ]:
# extract a subset of columns
# the first ":" indicates that we'll take all rows

df_subset = df_seatac.loc[:, ["DATE", "AWND", "PRCP", "TAVG", "TMAX", "TMIN"]]
display(df_subset)

In [ ]:
# You can use : to indicate range of columns

df_subset = df_seatac.loc[:, "TAVG":"TMIN"]
display(df_subset)

In [ ]:
# select rows by Boolean condition
# NOTE the implicit conversion of string to date

df_subset = df_seatac.loc[df_seatac["DATE"] >= "2024-06-01", :]
df_subset.head()

In [ ]:
# select row by position

df_subset = df_seatac.iloc[:100, :]
df_subset.tail()

### Additional column operations

In [ ]:
# Assign new column (temperature range)
# Note that this mutate (modify) the original dataframe

df_seatac["TRNG"] = df_seatac["TMAX"] - df_seatac["TMIN"]
display(df_seatac)

In [ ]:
# rename columns

df_renamed = df_seatac.rename(columns={"LATITUDE": "LAT", "LONGITUDE": "LON"})
df_renamed.sample(5)

In [ ]:
# Set Date as index

df_dated = df_seatac.set_index("DATE")
df_dated.head(4)

In [ ]:
# Reset the index into a regular column
# NOTE: to just "throw away" the index, use drop=True

df_indexed = df_seatac.reset_index()
display(df_indexed)

### Additional row operations

In [ ]:
# arrange data by increasing precipitations
df_sorted = df_seatac.sort_values("PRCP")
display(df_sorted)

### Copy-on-write behaviors

In [ ]:
# .value attributes fetch READ-ONLY numpy arrays
df_seatac.index.values = 2

In [ ]:
# subsetting creates copy
name = df_seatac["NAME"]
name.iloc[1] = "SEATAC"
display(name.head())
display(df_seatac.head())

In [ ]:
# chainned assignment will not work
df_seatac["DATE"].iloc[1] = pd.Timestamp("2025-01-02")

### Reduction (statistics) function in pandas

In [ ]:
# summary statistics for numerical columns
# Note the result is another DataFrame
seatac_stat = df_seatac.loc[:, ["PRCP", "TAVG", "TMAX", "TMIN"]].describe()
display(seatac_stat)

In [ ]:
# summary statistics for text columns
penguins_stat = df_penguins.loc[:, ["species", "island", "sex"]].describe()
display(penguins_stat)

In [ ]:
# count the occurence of various values
# the result is a multi-indexed pandas Series
penguins_count = df_penguins.loc[:, ["species", "island", "sex"]].value_counts()
display(penguins_count)

### Exercise 3

1. Subset the SeaTac DataFrame to include only the 5 columns `"DATE"`, `"PRCP"`, `"TAVG"`, `"TMAX"` and `"TMIN"` (in that order)
2. Retain only those rows whose precipitation (`"PRCP"`) is non-zero
3. Find the average precipitation over the days when precipitation (`"PRCP"`) is non-zero. Find also the maximum precipitation.
4. Sort the resulting DataFrame by average temperature (`"TAVG"`) in decreasing order, and thus find the precipation on the hottest day among the days with non-zero precipitation
5. You’ll notice that the data still carries the original index. Reset the index so that it starts from 0 and increments by 1 per row

In [ ]:
# Your codes for part 1

In [ ]:
# Your codes for part 2

In [ ]:
# Your codes for part 3

In [ ]:
# Your codes for part 4

In [ ]:
# Your codes for part 5

### Plotting pandas dataframe

In [ ]:
# panda Series can be used as arguments to matplotlib plot functions
# to specify where datetime ticks are and how to display them
# use tools from matplotlib.dates

fig = plt.figure()
ax = fig.add_subplot()

ax.set_xlabel("Dates in 2024")
ax.set_ylabel("Temperature (°C)")
ax.set_title("Tenperature at SeaTac airport")

locator = mdates.MonthLocator(bymonthday=15)
formatter = mdates.DateFormatter("%b")

ax.set_xlim(pd.to_datetime(["2024-01-01", "2024-12-31"]))
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

ax.plot(df_seatac["DATE"],df_seatac["TAVG"])

plt.show()

### Saving dataframe to file

In [ ]:
# write to csv file
seatac_stat.to_csv("seatac_stat.csv")

In [ ]:
# write to excel file
seatac_stat.to_excel("seatac_stat.xlsx")

In [ ]:
# write multiple DataFrames to same file (different sheet)
with pd.ExcelWriter("stats_examples.xlsx") as xlsx:
    seatac_stat.to_excel(xlsx, sheet_name="SeaTac")
    penguins_stat.to_excel(xlsx, sheet_name="Penguins")

## Pandas data type

### Convert datatype

In [ ]:
# start with a pandas series of floating point
numbers = pd.Series([1.0, 3.0, 2.0, np.nan, 4.0])
numbers

In [ ]:
# convert to string
numbers.astype("string")

In [ ]:
# convert to int
numbers.astype("Int64")

In [ ]:
# convert to float (no effect)
numbers.astype("float")

### Missing data: `np.nan`, `pd.NaT`, and `pd.NA`

In [ ]:
# np.nan and np.NaT are treated as falsy
print(
    bool(np.nan),
    bool(pd.NaT)
)

In [ ]:
# pd.NA will however raise an error when trying to convert to bool
bool(pd.NA)

In [ ]:
# check nullish via .isna()
print(
    pd.isna(None),
    pd.isna(np.nan),
    pd.isna(pd.NaT),
    pd.isna(pd.NA)
)

In [ ]:
# there's missing data in the SeaTac dataset
df_seatac["PGTM"].isna().sum()

In [ ]:
# nonetheless summary statistics can be calculated
# by ignoring the NA value
df_seatac["PGTM"].mean()

### String data type

In [ ]:
# concantenate values from all rows at once
penguin_class = df_penguins["island"] + "_" + df_penguins["species"]
penguin_class

In [ ]:
# use .str.contain to check substring
penguin_dream = penguin_class.str.contains("Dream")
penguin_dream

In [ ]:
df_penguins.loc[penguin_dream, :]

In [ ]:
# replace island name with shorthand
island_codes = df_penguins["island"].replace("Dream", "D").replace("Torgersen", "T").replace("Biscoe", "B")
island_codes

### Categorical data type

In [ ]:
# convert data to categorical type
df_penguins["island"] = df_penguins["island"].astype("category")
df_penguins["species"] = df_penguins["species"].astype("category")
df_penguins

In [ ]:
# recode (and possibly combine) categories
df_Biscoe = df_penguins["island"].map(
    {"Biscoe": "Biscoe", "Dream": "not_Biscoe", "Torgersen": "not_Biscoe"}
).astype("category")
df_Biscoe

### Date and datetime datatype

In [ ]:
# convert scalar string to timestamp

pd.Timestamp("2017-05-24 13:00:00")

In [ ]:
# convert a list of string into pandas datetime
pd.to_datetime(["2024-01-05", "2024-01-07", "2024-01-09"])

In [ ]:
pd.Series(pd.to_datetime(["2024-01-05", "2024-01-07", "2024-01-09"]))

In [ ]:
# create a regular list of timestamp
# use freq to specify interval between time points
pd.date_range("2024-01-03 10:00", "2024-03-04", freq="15D")

In [ ]:
# create a regular list of timestamp
# specify the number of time points rather than an endtime
pd.date_range("2024-01-03", periods=10, freq="2D")

In [ ]:
# Create a pandas Series of datetime
# use start, stop and periods to specify time points
pd.Series(pd.date_range("2024-01-01", "2024-02-01", periods=15))

In [ ]:
# a datetime range and include a few NaT's
drange = pd.Series(pd.date_range("2024-01-01", periods=12, freq="1MS"))
drange.iloc[[5, 8]] = pd.NaT
display(drange)

In [ ]:
# extract years
drange.dt.year

In [ ]:
# extract day of year
drange.dt.dayofyear

In [ ]:
# extract month
drange.dt.month

### Timezones

In [ ]:
# setting timezone directly when creating datetime Series

tz_London = pytz.timezone("Europe/London")
tzrng = pd.Series(pd.date_range(
    "2024-01-14 9:56:33", periods=10, freq="D", tz=tz_London
))
tzrng

In [ ]:
# augment original data with new timezone
# NOTE that the timezone is added without modifying the datetime value
drng = pd.date_range("3/6/2012 00:00", periods=15, freq="D")
tzrng2 = drng.tz_localize("America/New_York")
tzrng2

In [ ]:
# convert from one timezone to another
tzrng3 = tzrng2.tz_convert("UTC")
tzrng3

### Exercise 4

1. Create a pandas Series of datetime that spans 2024, and contain the first day of every month at 10:00 AM and the 15th of every month at 2:00 PM. Make sure that:
    1. your pandas Series is sorted by increasing date
    1. your pandas Series has a row index that starts from 0 and increment 1 per row
2. Set the timezone of the Series to Seattle local time (which timezone will work?)
3. Extract the day of year from the resulting Series
   
**Hint: #1**: you can use `pd.concat([A, B])` to join two pandas Series into one

**Hint: #2**: look up the IANA time zones [here](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones)

In [ ]:
# Your code for part 1

In [ ]:
# Your code for part 2

In [ ]:
# Your code for part 3

## Pandas grouped and rolling reduction

### Grouped reduction

In [ ]:
# Create a grouped DataFrame and calculated mean
df_penguins.loc[:, ["species", "flipper_length_mm", "body_mass_g"]].groupby("species").mean()

In [ ]:
# It is possible to group by multiple columns
df_penguins.loc[:, ["species", "island", "flipper_length_mm", "body_mass_g"]].groupby(["species", "island"]).mean()

### Rolling reduction

In [ ]:
# rolling average over periods of 14 data points (= 2 week)
rolling_PRCP = df_seatac["PRCP"].rolling(14, center=True).mean()

In [ ]:
# compare raw data with rolling average

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(df_seatac["DATE"], df_seatac["PRCP"], label="raw data")
ax.plot(df_seatac["DATE"], rolling_PRCP, lw=2, ls="--", label="rolling average")

ax.legend()

plt.show()

### Exercise 5

**Groupby and rolling**

1. Calculate the monthly average temperature from the SeaTac dataset, and store the results in variable `TAVG_monthly`
2. Calculate the one-week rolling average temperature from the SeaTac dataset, and store the results in variable `TAVG_rolling`
3. Plot `TAVG_monthly` and `TAVG_rolling` on the same axes, using the starter code as template. Your x-axis should display abbreviated months as ticks (i.e., "Jan", "Feb", etc.). Your `TAVG_monthly` should be presented as vertical bars, and your `TAVG_rolling` should be presented as a line.

In [ ]:
# your code for part 1

In [ ]:
# your code for part 2

In [ ]:
# part 3, with starter code

fig = plt.figure()
ax = fig.add_subplot()

ax.set_xlabel("Months of 2024")
ax.set_ylabel("Temperature (°C)")
ax.set_title("Tenperature at SeaTac airport")


plt.legend()

plt.show()

## Xarray

### Loading and inspecting a netcdf file (xarray Dataset)

**NOTE**: data downloaded from https://www.ncei.noaa.gov/access/world-ocean-atlas-2023/bin/woa23.pl

In [ ]:
woa23 = xr.open_dataset("data/woa23_decav_t00_01.nc", decode_times=False)
display(woa23)

In [ ]:
# extract a particular data variable
woa23t = woa23["t_an"]
display(woa23t)

In [ ]:
# extract one of the coordinates
woa23.coords["lat"]

In [ ]:
# obtain the underlying numpy array of the above coordinates
woa23.coords["lat"].values

In [ ]:
# extract the dimension names
woa23.dims

In [ ]:
# get the size information of data
woa23.sizes

In [ ]:
# get the global metadata
woa23.attrs

In [ ]:
# get the metadata of one data variable
woa23["t_an"].attrs

### Selecting using `[]`, `.sel()`, and `.isel()`

In [ ]:
# extract the DataArray corresonding to objectively analyzed mean
woa23t = woa23["t_an"]

In [ ]:
# selecting both latitude and longitude by coordinate labels
woa_point = woa23t.sel(lat=50.5, lon=-150.5)
display(woa_point)
display(woa_point.values)

In [ ]:
# approximate matching by coordinate labels
woa_point = woa23t.sel(lat=80, lon=-150, method="nearest")
display(woa_point)
display(woa_point.values)

In [ ]:
# use squeeze to remove dimensions with only 1 index
woa_squeezed = woa_point.squeeze()
display(woa_squeezed)

In [ ]:
# slicing by coordinate labels
# Note that when using .sel, slicing is endpoint inclusive
AmPac = woa23t.sel(lat=slice(30.5, 50.5), lon=slice(-150, -130))
display(AmPac)

In [ ]:
# selection by boolean
woa_subset = woa23t.sel(lat = woa23t.coords["lat"] % 5 == 2.5, lon = woa23t.coords["lon"] % 5 == 2.5)
display(woa_subset)

In [ ]:
# average over depth. Note that nan is ignored
AmPac = woa23t.sel(lat=slice(30.5, 50.5), lon=slice(-150, -130))
AmPac.mean(["depth", "time"])

### Converting between xarray and pandas

In [ ]:
# convert 
woa23 = xr.open_dataset("data/woa23_decav_t00_01.nc", decode_times=False)

# extract only data variables that don't depend on nbounds
woa23ts = woa23[["t_an", "t_mn", "t_dd", "t_sd", "t_se", "t_oa", "t_gp", "t_sdo", "t_sea"]]

# extract only the surface depth, and squeeze extra dimensions
woa23ts_surf = woa23ts.sel(depth=0, method="nearest").squeeze()

# convert to pandas dataframe
woa23_pd = woa23ts_surf.to_dataframe()
display(woa23_pd)

# reset the multi-index dataframe into regular one
woa23_pd2 = woa23_pd.reset_index()
display(woa23_pd2)

In [ ]:
# reverse the operation
woa23_xr = woa23_pd2.set_index(["lat", "lon"]).to_xarray()

In [ ]:
woa23_xr

### Writing netcdf file

In [ ]:
# start with the SeaTac dataframe and turn it into a dataset
df_seatac = pd.read_csv("data/NOAA_SeaTac_weather_2024.csv", parse_dates = ["DATE"]).convert_dtypes()
seatac_sub = df_seatac[["LATITUDE", "LONGITUDE", "ELEVATION", "DATE", "PRCP", "TAVG"]]
seatac_xr = seatac_sub.set_index(["LATITUDE", "LONGITUDE", "ELEVATION", "DATE"]).to_xarray()

In [ ]:
display(seatac_xr)

In [ ]:
# write this Dataset to file, specifying compression
seatac_xr.to_netcdf(
    "SeaTac_weather.nc", 
    format="NETCDF4", 
    encoding = {"PRCP": {"zlib": True, "complevel": 9}, "TAVG": {"zlib": True, "complevel": 9}}
)

## Cartopy

### Map projections

In [ ]:
# example data: surface temperature from woa23
woa23 = xr.open_dataset("data/woa23_decav_t00_01.nc", decode_times=False)
woa23_surf = woa23.sel(depth=0, method="nearest").squeeze()

surf_temp = woa23_surf["t_an"].values
surf_lat = woa23_surf.coords["lat"].values
surf_lon = woa23_surf.coords["lon"].values

print(surf_temp.shape, surf_lat.shape, surf_lon.shape)

In [ ]:
# plot annual temperature on the global scale

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(projection=ccrs.InterruptedGoodeHomolosine())

mesh = ax.pcolormesh(surf_lon, surf_lat, surf_temp, cmap=cmo.thermal, transform=ccrs.PlateCarree())
cb = fig.colorbar(mesh)
cb.set_label("annual temperature (°C)")

plt.show()

### Geographical features

In [ ]:
# adding coastline, land, and border features

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(projection=ccrs.InterruptedGoodeHomolosine())

#### << START NEW CONTENT ####
ax.coastlines()
ax.add_feature(cfeature.LAND, color="whitesmoke")
ax.add_feature(cfeature.BORDERS, edgecolor="brown")
#### >> END NEW CONTENT ####

mesh = ax.pcolormesh(surf_lon, surf_lat, surf_temp, cmap=cmo.thermal, transform=ccrs.PlateCarree())
cb = fig.colorbar(mesh)
cb.set_label("annual temperature (°C)")

plt.show()

In [ ]:
# custom NaturalEarth feature

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(projection=ccrs.InterruptedGoodeHomolosine())

#### << START NEW CONTENT ####
roads = cfeature.NaturalEarthFeature(
    category='cultural',
    name='urban_areas',
    scale='50m',
)
ax.coastlines()
ax.add_feature(roads, color="darkgoldenrod")
#### >> END NEW CONTENT ####

mesh = ax.pcolormesh(surf_lon, surf_lat, surf_temp, cmap=cmo.thermal, transform=ccrs.PlateCarree())
cb = fig.colorbar(mesh)
cb.set_label("annual temperature (°C)")

plt.show()

### Restricting extent

In [ ]:
# restricting the geographical extent of the plot

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(projection=ccrs.PlateCarree())

#### << START NEW CONTENT ####
# set_extent format: (x0, x1, y0, y1)
ax.set_extent([-75, 0, 30, 60], crs=ccrs.PlateCarree())
#### >> END NEW CONTENT ####

ax.coastlines()

mesh = ax.pcolormesh(surf_lon, surf_lat, surf_temp, cmap=cmo.thermal, transform=ccrs.PlateCarree())
cb = fig.colorbar(mesh)
cb.set_label("annual temperature (°C)")

plt.show()

### Gridlines

In [ ]:
# including gridlines for lat and lon

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(projection=ccrs.Robinson())

ax.coastlines()

mesh = ax.pcolormesh(surf_lon, surf_lat, surf_temp, cmap=cmo.thermal, transform=ccrs.PlateCarree())
cb = fig.colorbar(mesh)
cb.set_label("annual temperature (°C)")

#### << START NEW CONTENT ####
ax.gridlines(
    crs=ccrs.PlateCarree(), draw_labels=True,
    linewidth=1, color='darkgray', linestyle='--'
)
#### >> END NEW CONTENT ####

plt.show()

In [ ]:
# further customize gridline properties
# NOTE: the current interface is somewhat buggy....

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(projection=ccrs.Robinson())

ax.coastlines()

mesh = ax.pcolormesh(surf_lon, surf_lat, surf_temp, cmap=cmo.thermal, transform=ccrs.PlateCarree())
cb = fig.colorbar(mesh)
cb.set_label("annual temperature (°C)")

#### << START NEW CONTENT ####
gl = ax.gridlines(
    crs=ccrs.PlateCarree(), draw_labels=True,
    linewidth=1, color='darkgray', linestyle='--'
)
gl.left_labels = True
gl.right_labels = False
gl.top_labels = True
gl.bottom_labels = False
gl.xlocator = mticker.FixedLocator([-180, -90, -45, 0, 45, 90, 180])
gl.ylocator = mticker.FixedLocator([-60, -30, -15, 0, 15, 30, 60])
gl.xformatter = cmgrid.LONGITUDE_FORMATTER
gl.yformatter = cmgrid.LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10, 'color': 'red'}
gl.ylabel_style = {'size': 12, 'color': 'gray', 'weight': 'bold'}
#### >> END NEW CONTENT ####

plt.show()

### Exercise 6

_**Code writing #5.**_ xarray and cartopy

1. Load mercatorbiomer4v2r1_global_mean_pft_2024_sub.nc in the data folder as a xarray Dataset named `mercatorbio`
2. Subset / reduce the Dataset as follows:
    1. Look only at the chlorophyll (chl) variable
    2. Look only at a depth of (closest to) 40 m
    3. Average over all months

    Assign the resulting DataArray to the variable `mercator_chl`

3. Plot the resulting chlorophyll concentration across the globe using cartopy


In [ ]:
# Your code for part 1

In [ ]:
# Your code for part 2

In [ ]:
# Your code for part 3